In [142]:
import pandas as pd
import numpy as np
import re

In [143]:
df = pd.read_excel('integrated_data.xlsx')

In [144]:
def peek(df, cols=None, n=5, title=None):
    if title: print(f"\n {title}")
    if cols is not None:
        cols = [c for c in cols if c in df.columns]
        display(df[cols].head(n))
    else:
        display(df.head(n))

def report_missing(df, cols=None, title=None):
    if title: print(f"\nMissing ratio — {title}")
    s = df[cols] if cols else df
    out = (s.isna().mean()*100).round(2).sort_values(ascending=False)
    display(out.to_frame("missing_%").head(20))

def compare_size(dfa, dfb, label=""):
    a, b = dfa.shape, dfb.shape
    print(f"\n{label} | before: {a} -> after: {b} | rows: {b[0]-a[0]}, cols: {b[1]-a[1]}")

def value_counts_top(df, col, k=10, title=None):
    if col not in df.columns: 
        print(f"(skip) '{col}' không có trong DataFrame"); 
        return
    if title: print(f"\nTop {k} '{col}' — {title}")
    display(df[col].value_counts(dropna=False).head(k).to_frame("count"))


In [145]:
df0 = df.copy()
core_cols = [c for c in ["id","name","brand","category_l1","price","list_price","discount_percent",
                        "rating_average","quantity_sold_value","image_path"] if c in df0.columns]
print("Shape:", df0.shape)
peek(df0, core_cols, title="Core columns (preview)")
report_missing(df0, core_cols, title="Core columns")

Shape: (500, 10)

 Core columns (preview)


,id,name,brand,category_l1,price,rating_average,quantity_sold_value,image_path
0,270975124,đien thoai oppo a58 6gb/128gb - hang chinh hang,oppo,phone,3890000,5.0,25120,images/phone/product_270975124.jpg
1,184059211,apple iphone 13,apple,phone,11350000,5.0,6815,images/phone/product_184059211.jpg
2,277007015,đien thoai xiaomi redmi note 14 6gb/128gb - ha...,xiaomi,phone,3890000,5.0,6256,images/phone/product_277007015.jpg
3,275510578,đien thoai oppo a79 5g (8gb/256gb) - hang chin...,oppo,phone,5690000,5.0,5453,images/phone/product_275510578.jpg
4,84866683,camera ip wifi trong nha ezviz mini cs-c1c-b 1...,ezviz,camera,370000,4.7,4070,images/camera/product_84866683.jpg



Missing ratio — Core columns


,missing_%
id,0.0
name,0.0
brand,0.0
category_l1,0.0
price,0.0
rating_average,0.0
quantity_sold_value,0.0
image_path,0.0


Chuẩn hoá text

In [146]:
df1 = df0.copy()

for c in ["name","brand","category_l1"]:
    if c in df1.columns:
        df1[c] = df1[c].astype(str).str.strip()
compare_size(df0, df1, "Step: strip text")
peek(df1, ["name","brand","category_l1"])


Step: strip text | before: (500, 10) -> after: (500, 10) | rows: 0, cols: 0


,name,brand,category_l1
0,đien thoai oppo a58 6gb/128gb - hang chinh hang,oppo,phone
1,apple iphone 13,apple,phone
2,đien thoai xiaomi redmi note 14 6gb/128gb - ha...,xiaomi,phone
3,đien thoai oppo a79 5g (8gb/256gb) - hang chin...,oppo,phone
4,camera ip wifi trong nha ezviz mini cs-c1c-b 1...,ezviz,camera


discount_abs & has_promo

In [147]:
df2 = df1.copy()

if {"list_price","price"}.issubset(df2.columns):
    with np.errstate(invalid="ignore"):
        df2["discount_abs"] = (df2["list_price"] - df2["price"]).where(
            (df2["list_price"].notna()) & (df2["price"].notna())
        )
else:
    df2["discount_abs"] = np.nan

has_disc_pct = df2["discount_percent"].fillna(0).gt(0) if "discount_percent" in df2.columns else False
has_disc_abs = df2["discount_abs"].fillna(0).gt(0)
df2["has_promo"] = (has_disc_pct | has_disc_abs).astype(int)

compare_size(df1, df2, "Step: discount_abs & has_promo")
peek(df2, ["price","list_price","discount_percent","discount_abs","has_promo"])
value_counts_top(df2, "has_promo", title="Has promo")


Step: discount_abs & has_promo | before: (500, 10) -> after: (500, 12) | rows: 0, cols: 2


,price,discount_abs,has_promo
0,3890000,NaN,0
1,11350000,NaN,0
2,3890000,NaN,0
3,5690000,NaN,0
4,370000,NaN,0



Top 10 'has_promo' — Has promo


,count
has_promo,
0,500


has_image

In [148]:
df3 = df2.copy()

if "image_path" in df3.columns:
    df3["has_image"] = df3["image_path"].astype(str).str.strip().ne("").astype(int)
else:
    df3["has_image"] = 0
compare_size(df2, df3, "Step: has_image")
value_counts_top(df3, "has_image")


Step: has_image | before: (500, 12) -> after: (500, 13) | rows: 0, cols: 1


,count
has_image,
1,500


đặc trưng từ tên

In [149]:
df4 = df3.copy()
if "name" in df4.columns:
    n = df4["name"].fillna("")
    df4["name_len"]   = n.str.len()
    df4["name_words"] = n.str.split().str.len()
    def has_kw(pattern): 
        return n.str.contains(pattern, flags=re.IGNORECASE, regex=True, na=False).astype(int)
    df4["kw_pro"]        = has_kw(r"\bpro\b")
    df4["kw_max"]        = has_kw(r"\bmax\b")
    df4["kw_ultra"]      = has_kw(r"\bultra\b")
    df4["kw_5g"]         = has_kw(r"\b5g\b")
    df4["kw_bluetooth"]  = has_kw(r"\bbluetooth\b")
    df4["kw_gaming"]     = has_kw(r"\bgaming\b")
    df4["kw_chinh_hang"] = has_kw(r"ch(i|í)nh\s*h(ă|a)ng|chinh\s*hang")

compare_size(df3, df4, "Step 4: name features")
peek(df4, ["name","name_len","name_words","kw_pro","kw_max","kw_ultra","kw_5g","kw_bluetooth","kw_gaming","kw_chinh_hang"])


Step 4: name features | before: (500, 13) -> after: (500, 22) | rows: 0, cols: 9


/var/folders/g0/ncn4nyg10pn4_70kw70z_9bh0000gn/T/ipykernel_64152/2845341920.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return n.str.contains(pattern, flags=re.IGNORECASE, regex=True, na=False).astype(int)


,name,name_len,name_words,kw_pro,kw_max,kw_ultra,kw_5g,kw_bluetooth,kw_gaming,kw_chinh_hang
0,đien thoai oppo a58 6gb/128gb - hang chinh hang,47,9,0,0,0,0,0,0,1
1,apple iphone 13,15,3,0,0,0,0,0,0,0
2,đien thoai xiaomi redmi note 14 6gb/128gb - ha...,59,11,0,0,0,0,0,0,1
3,đien thoai oppo a79 5g (8gb/256gb) - hang chin...,52,10,0,0,0,1,0,0,1
4,camera ip wifi trong nha ezviz mini cs-c1c-b 1...,68,13,0,0,0,0,0,0,1


brand_freq

In [150]:
df5 = df4.copy()
if "brand" in df5.columns:
    brand_counts = df5["brand"].fillna("").str.strip().value_counts(dropna=False)
    df5["brand_freq"] = df5["brand"].map(brand_counts).fillna(0).astype(int)
else:
    df5["brand_freq"] = 0

compare_size(df4, df5, "Step 5: brand_freq")
peek(df5, ["brand","brand_freq"])


Step 5: brand_freq | before: (500, 22) -> after: (500, 23) | rows: 0, cols: 1


,brand,brand_freq
0,oppo,8
1,apple,13
2,xiaomi,27
3,oppo,8
4,ezviz,18


rating_level & qty_bucket

In [151]:
df6 = df5.copy()

def mk_rating_level(x):
    if pd.isna(x): return "unknown"
    if x < 3:      return "low"
    if x < 4:      return "med"
    return "high"

df6["rating_level"] = df6["rating_average"].map(mk_rating_level) if "rating_average" in df6.columns else "unknown"

if "quantity_sold_value" in df6.columns:
    q = df6["quantity_sold_value"].fillna(0)
    try:
        q1, q2 = q.quantile([0.5, 0.85])
    except Exception:
        q1, q2 = 1, 10
    conds = [q.eq(0), (q>0)&(q<=q1), (q>q1)&(q<=q2), (q>q2)]
    df6["qty_bucket"] = np.select(conds, ["zero","low","mid","high"], default="low")
else:
    df6["qty_bucket"] = "unknown"

compare_size(df5, df6, "Step 6: rating_level & qty_bucket")
value_counts_top(df6, "rating_level", title="Rating level")
value_counts_top(df6, "qty_bucket",   title="Quantity bucket")


Step 6: rating_level & qty_bucket | before: (500, 23) -> after: (500, 25) | rows: 0, cols: 2

Top 10 'rating_level' — Rating level


,count
rating_level,
low,276
high,214
med,10



Top 10 'qty_bucket' — Quantity bucket


,count
qty_bucket,
low,258
mid,168
high,74


In [152]:
df_trans = df6.copy()
print("\nTransformation hoàn tất. df_trans shape:", df_trans.shape)

# Xem nhanh toàn bộ feature mới
new_feats = [c for c in ["discount_abs","has_promo","has_image","name_len","name_words",
                        "kw_pro","kw_max","kw_ultra","kw_5g","kw_bluetooth","kw_gaming","kw_chinh_hang",
                        "brand_freq","rating_level","qty_bucket"] if c in df_trans.columns]
peek(df_trans, ["id","name","brand","category_l1","price","list_price","discount_percent"] + new_feats,
     title="Tổng hợp cột gốc + feature mới (preview)")
report_missing(df_trans, new_feats, title="Missing của các feature mới")


Transformation hoàn tất. df_trans shape: (500, 25)

 Tổng hợp cột gốc + feature mới (preview)


,id,name,brand,category_l1,price,discount_abs,has_promo,has_image,name_len,name_words,kw_pro,kw_max,kw_ultra,kw_5g,kw_bluetooth,kw_gaming,kw_chinh_hang,brand_freq,rating_level,qty_bucket
0,270975124,đien thoai oppo a58 6gb/128gb - hang chinh hang,oppo,phone,3890000,NaN,0,1,47,9,0,0,0,0,0,0,1,8,high,high
1,184059211,apple iphone 13,apple,phone,11350000,NaN,0,1,15,3,0,0,0,0,0,0,0,13,high,high
2,277007015,đien thoai xiaomi redmi note 14 6gb/128gb - ha...,xiaomi,phone,3890000,NaN,0,1,59,11,0,0,0,0,0,0,1,27,high,high
3,275510578,đien thoai oppo a79 5g (8gb/256gb) - hang chin...,oppo,phone,5690000,NaN,0,1,52,10,0,0,0,1,0,0,1,8,high,high
4,84866683,camera ip wifi trong nha ezviz mini cs-c1c-b 1...,ezviz,camera,370000,NaN,0,1,68,13,0,0,0,0,0,0,1,18,high,high



Missing ratio — Missing của các feature mới


,missing_%
discount_abs,100.0
has_promo,0.0
has_image,0.0
name_len,0.0
name_words,0.0
kw_pro,0.0
kw_max,0.0
kw_ultra,0.0
kw_5g,0.0
kw_bluetooth,0.0


In [153]:
df = df_trans.copy()

In [154]:
df.to_excel("transformation_data.xlsx", index=False)
print("Saved transformation_data.xlsx")

Saved transformation_data.xlsx
